Import Dependencies

In [14]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_URI = os.getenv("DATABASE_URI")

print("Key loaded:", bool(OPENAI_API_KEY))
print("DB URI:", bool(DATABASE_URI))

Key loaded: True
DB URI: True


Setup LangChain and connect to PostgreSql

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.utilities import SQLDatabase

In [10]:
#PostgreSql URI - URI -encoded password
DATABASE_URI = os.getenv("DATABASE_URI")

#connect to the database
db = SQLDatabase.from_uri(DATABASE_URI)

Initialize LLM and SQl Chain

In [5]:
llm = ChatOpenAI(temperature=0,model='gpt-4')
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True)

C:\Users\marty\AppData\Local\Temp\ipykernel_11036\1780378031.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0,model='gpt-4')


In [6]:
#Ask natural Langauge Questions
question = 'What is the average charge for smokers?'
response = db_chain(question)
print('Response',response)

C:\Users\marty\AppData\Local\Temp\ipykernel_11036\1979368892.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = db_chain(question)




> Entering new SQLDatabaseChain chain...
What is the average charge for smokers?
SELECT AVG("charges") FROM medical_expenses WHERE "smoker" = 'yes'
SQLResult: [(32050.23183153285,)]
The average charge for smokers is approximately 32050.23.
> Finished chain.
Response {'query': 'What is the average charge for smokers?', 'result': 'The average charge for smokers is approximately 32050.23.'}


In [7]:
while True:
    q = input("Ask a question (or type 'exit'): ")
    if q.lower() == "exit":
        break
    try:
        result = db_chain.run(q)
        print("Answer:", result)
    except Exception as e:
        print("Error:", e)

Ask a question (or type 'exit'):  what is the average charge cost for a female smoker over 40?


C:\Users\marty\AppData\Local\Temp\ipykernel_11036\943830914.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = db_chain.run(q)




> Entering new SQLDatabaseChain chain...
what is the average charge cost for a female smoker over 40?
SELECT AVG("charges") FROM medical_expenses WHERE "sex" = 'female' AND "smoker" = 'yes' AND "age" > 40
SQLResult: [(34517.67366452831,)]
The average charge cost for a female smoker over 40 is approximately 34517.67.
> Finished chain.
Answer: The average charge cost for a female smoker over 40 is approximately 34517.67.


Ask a question (or type 'exit'):  what region has the highest average charges?




> Entering new SQLDatabaseChain chain...
what region has the highest average charges?
SELECT "region", AVG("charges") as average_charges
FROM medical_expenses
GROUP BY "region"
ORDER BY average_charges DESC
LIMIT 1
SQLResult: [('southeast', 14735.411437609895)]
The region with the highest average charges is southeast.
> Finished chain.
Answer: The region with the highest average charges is southeast.


...

Ask a question (or type 'exit'):  does having a high bi and being a smoking affect your charges




> Entering new SQLDatabaseChain chain...
does having a high bi and being a smoking affect your charges
SELECT "smoker", AVG("charges") as average_charges
FROM medical_expenses
WHERE "bmi" > 30
GROUP BY "smoker"
LIMIT 5;
SQLResult: [('no', 8853.277293600706), ('yes', 41692.808991527796)]
Yes, having a high BMI and being a smoker does affect your charges. On average, smokers with a high BMI have charges of approximately 41692.81, while non-smokers with a high BMI have charges of approximately 8853.28.
> Finished chain.
Answer: Yes, having a high BMI and being a smoker does affect your charges. On average, smokers with a high BMI have charges of approximately 41692.81, while non-smokers with a high BMI have charges of approximately 8853.28.


Ask a question (or type 'exit'):  what is are the average charges for a smoker over the age of 50 and younger then 85?




> Entering new SQLDatabaseChain chain...
what is are the average charges for a smoker over the age of 50 and younger then 85?
SELECT AVG("charges") FROM medical_expenses WHERE "smoker" = 'yes' AND "age" > 50 AND "age" < 85
SQLResult: [(38820.22308234375,)]
The average charges for a smoker over the age of 50 and younger than 85 is approximately 38820.22.
> Finished chain.
Answer: The average charges for a smoker over the age of 50 and younger than 85 is approximately 38820.22.


Ask a question (or type 'exit'):  exit
